In [1]:
# prepend parent dir to import from local pcsp
import sys

sys.path.insert(0, '../')

In [9]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from vflow import Vset, init_args, dict_to_df
from functools import partial
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score, explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ind = {}


def top_n_features(X, Y, n, i):
    if i not in ind:
        corr = np.abs(np.apply_along_axis(lambda x: np.corrcoef(x, Y[:, i])[0, 1], 0, X))
        ind[i] = np.argsort(corr[~np.isnan(corr)])[::-1][:n]
    return X[:, ind[i]]


def pca(X, n):
    return PCA(n_components=n, copy=True).fit(X).transform(X)


In [4]:
# load data
data_dir = "../../_other/data/"
X = np.asarray(pd.read_csv(data_dir + "fit_feat.txt", sep=" ", error_bad_lines=False))
Y = np.asarray(pd.read_csv(data_dir + "resp_dat.txt", sep=" ", error_bad_lines=False))

In [5]:
np.random.seed(14)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=14)
X_train, X_test, y_train, y_test = init_args((X_train, X_test, y_train, y_test),
                                             names=['X_train', 'X_test', 'y_train', 'y_test'])

# split y_train by voxel and extract top 500 correlated features per voxel
voxel_extract_funcs = [partial(lambda x, y, i: (top_n_features(x, y, 20, i), y[:, i]), i=i) for i in range(20)]
voxel_extract_set = Vset(name='voxel_extract', modules=voxel_extract_funcs, output_matching=True)
X_trains, y_trains = voxel_extract_set(X_train, y_train)
X_tests, y_tests = voxel_extract_set(X_test, y_test)

/Users/rush/anaconda3/envs/vflow/lib/python3.6/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/rush/anaconda3/envs/vflow/lib/python3.6/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [6]:
# modeling
modeling_set = Vset(name='modeling', modules=[LassoCV()], module_keys=["Lasso"])
modeling_set.fit(X_trains, y_trains)

In [7]:
preds = modeling_set.predict(X_trains)

hard_metrics_set = Vset(name='hard_metrics', modules=[r2_score, explained_variance_score],
                             module_keys=["R2", "EV"])
hard_metrics = hard_metrics_set.evaluate(y_trains, preds)

In [10]:
dict_to_df(hard_metrics)

,init-voxel_extract,voxel_extract,init-modeling,init-modeling,init-modeling,modeling,hard_metrics,out
0,y_train,voxel_extract_0,X_train,X_train,y_train,Lasso,R2,0.170800
1,y_train,voxel_extract_1,X_train,X_train,y_train,Lasso,R2,0.215821
2,y_train,voxel_extract_2,X_train,X_train,y_train,Lasso,R2,0.147640
3,y_train,voxel_extract_3,X_train,X_train,y_train,Lasso,R2,0.166916
4,y_train,voxel_extract_4,X_train,X_train,y_train,Lasso,R2,0.161540
5,y_train,voxel_extract_5,X_train,X_train,y_train,Lasso,R2,0.191273
6,y_train,voxel_extract_6,X_train,X_train,y_train,Lasso,R2,0.212912
7,y_train,voxel_extract_7,X_train,X_train,y_train,Lasso,R2,0.214804
8,y_train,voxel_extract_8,X_train,X_train,y_train,Lasso,R2,0.234716
9,y_train,voxel_extract_9,X_train,X_train,y_train,Lasso,R2,0.039008
